<a href="https://colab.research.google.com/github/uttarakhatri/Sleep-stage-EEG-transitions/blob/main/Recurrence_threshold_chooser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#installing packages
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-cairo python3-matplotlib
import graph_tool.all as gt

#importing modules
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.spatial import distance
import pandas as pd
import collections
from mpl_toolkits import mplot3d
%matplotlib inline
import seaborn as sns

In [ ]:
sub_rn_thr= np.zeros((10,6,20))
def rnthr():

  for subj in range(1,11):
    print(subj)
    stop_stages= np.load('/content/drive/MyDrive/sleep/ISRUC_sleep_dataset/random_epochs/'+str(subj)+ '_1_random_60_epochs.npy')
    sub_w_stop= stop_stages[4]
    start= (sub_w_stop-1)[:20]
    sub1= np.load('/content/drive/My Drive/sleep/ISRUC_sleep_dataset/ICA/' + str(subj)+ '_ica_pc.npy')
    for channel in range(3,4):
      rec_thr_w=[]
      print((channel))
      for g in range(len(start)):
        #for each channel, each stage, RN analysis code
        #embedding
        tau=0
        da= sub1[channel][int(start[g]*6000):int((start[g]+1)*6000)]
        da= (da-min(da))/(max(da)-min(da))
        rel = sm.tsa.acf(da, nlags=75, fft=True)
        y= np.arange(0,75,1.0)
        for k in range(len(y)):
          if (rel[k]-(np.e**(-1)))<10**(-6):
            tau=k
            break
        le=int(len(da)-6*tau)
        dim= np.zeros((6,le))
        for i in range(6):
          for j in range(le):
              dim[i][j]=(da[j+i*tau])
        tra=dim.transpose()
        rec_matrix= np.zeros((le,le))
        rec_thr= np.arange(0.09,0.3,0.02)
        for c in range(len(rec_thr)):
          rec_matrix = (distance.cdist(tra,tra,'euclidean')<=rec_thr[c]) -(np.identity(le))
          g1= gt.Graph(directed=False)
          g1.add_edge_list(np.transpose(np.transpose(rec_matrix).nonzero()))
          l=gt.label_largest_component(g1)
          u = gt.GraphView(g1, vfilt = l)
          if (u.num_vertices()/le)>=0.95:
            #print(u.num_vertices()/le)
            #print(rec_thr[c])
            sub_rn_thr[subj-1][channel-3][g]=rec_thr[c]
            rec_thr_w.append(rec_thr[c])
            break
      print((np.mean(rec_thr_w), subj, channel))
      #sub_rn_thr[i-1][channel-3]= rec_thr_w


In [ ]:
rnthr()


In [ ]:
print(sub_rn_thr[0:10][0])

In [ ]:
for subje in range(10):
  plt.scatter([1],np.mean(sub_rn_thr[subje][0]))